In [1]:
# Loading all important Libraries
import numpy as np
import pickle
from keras.models import model_from_json
import pandas as pd
import datetime
import re
from prettytable import PrettyTable
import warnings
warnings.filterwarnings('ignore')

In [2]:
class final:
    def load_files(self): # This function loads all important files
        #https://machinelearningmastery.com/save-load-keras-deep-learning-models/
        json_file = open('model.json', 'r') #Loads model
        loaded_model_json = json_file.read()
        json_file.close()
        self.model = model_from_json(loaded_model_json)
        self.model.load_weights("model.h5") # Loads weights of model

        with open('access_enc.pkl','rb') as file:
            self.access_enc=pickle.load(file) #Loads label encoder for access

        with open('lang_enc.pkl','rb') as file:
            self.lang_enc=pickle.load(file) #Loads label encoder for language

        with open('spider_enc.pkl','rb') as file:
            self.agent_enc=pickle.load(file) #Loads label encoder for agent
        self.new_data=pd.read_csv('final_data.csv')    #Loads data which is required to make prediction 
    
    def find_access(self,page):#This function finds the client of the page for which we are making prediction
        k=max([i.start() for i in re.finditer('org_',page)])   #https://www.geeksforgeeks.org/python-all-occurrences-of-substring-in-string/
        if('all-access' in page[k:]):
            access='all_access'
        if('desktop' in page[k:]):
            access='desktop'
        if('mobile' in page[k:]): 
            access='mobile'
        k=access    
        access=self.access_enc.transform([access]).reshape(1,1)    
        return access,k 
    def find_lang(self,page): # This function finds language of the page for which we are making prediction
        index=page.find('.wikipedia')
        lang=page[index-1:index-3:-1][::-1]
        lang_dict={'de':'German','en':'English', 'es':'Spanish', 'fr':'French', 'ja':'Japanese', 'nt':'Media', 'ru':'Russian', 'zh':'Chinese'}
        language=lang_dict[lang]
        lang=self.lang_enc.transform([lang]).reshape(1,1)
        return lang,language
    def find_agent(self,page): #This page finds if the page was accessed by a spider or not.
        if('spider' in page):
            spider='spider'
        else:
            spider='non-spider' 
        k=spider  
        agent=self.agent_enc.transform([spider]).reshape(1,1)
        return agent,k
    def find_data(self,ind,date): 
        ''' This function returns the traffic on last 5 days on the page on which we are making prediction,
         this data is neccessary in order to make prediction and this data will be fed to loaded model.'''
        data=self.new_data.iloc[ind].values
        date1=datetime.date(2015,7,6)
        k=date.split('-')
        date2=datetime.date(int(k[0]),int(k[1]),int(k[2]))
        dif=(date2-date1).days
        data=np.log1p(data[dif+1:dif+6].astype(int))
        data=np.array(data).reshape(1,5,1)
        return data     
    def final_fun_1(self,ind,date): # This data takes index of the page and date as input for which we want to make prediction
        self.load_files()
        start=datetime.datetime.now()
        self.page=self.new_data['Page'].values[ind]
        access,access1=self.find_access(self.page)
        lang,language=self.find_lang(self.page)
        agent,agent1=self.find_agent(self.page)
        data=self.find_data(ind,date)
        predicted=int(np.round(np.expm1(self.model.predict([data,access,lang,agent])[0])[0]))
        
        x = PrettyTable()
        x = PrettyTable(["Client","Access", "Language",'Predicted','Time Taken'])
        x.add_row([agent1,access1,language,predicted,datetime.datetime.now()-start])

        print(x)
        #Prints the client,access,language,time taken and predicted traffic on the page
    def final_fun_2(self,set): #This function takes index,date and actual traffic on the page as input, can also take a list of multiple inputs
        self.load_files()
        x = PrettyTable()
        x = PrettyTable(["Client","Access", "Language",'Predicted','Actual','SMAPE','Time Taken'])
        for val in set:
            start=datetime.datetime.now()
            self.page=self.new_data['Page'].values[val[0]]
            access,access1=self.find_access(self.page)
            lang,language=self.find_lang(self.page)
            agent,agent1=self.find_agent(self.page)
            data=self.find_data(val[0],val[1])
            actual=int(val[2])
            predicted=int(np.round(np.expm1(self.model.predict([data,access,lang,agent],steps=1)[0])[0]))
            smape=np.abs(actual-predicted)/((actual+predicted)/2)

            x.add_row([agent1,access1,language,predicted,actual,np.round(smape,3),datetime.datetime.now()-start])
            #Print client,access,language,prediction,actual,time taken and SMAPE of the page
        print(x)

In the above final functions taken 10K pages to optimize the time and preprocessed them to test the model, it will work same on the whole data as well. Loaded the saved lstm model to predict and label encoded language, access and agent feature and used to transformed the test data feature.

In [8]:
final_object = final() #creating the object of final class

It is not possible to take all the features name or any particular page name so taken a index and date.

In [5]:
final_object.final_fun_1(8,'2015-07-07') #taken a random page index to test the model 

+--------+------------+----------+-----------+----------------+
| Client |   Access   | Language | Predicted |   Time Taken   |
+--------+------------+----------+-----------+----------------+
| spider | all_access | Chinese  |     5     | 0:00:06.107170 |
+--------+------------+----------+-----------+----------------+


In [10]:
data=pd.read_csv('final_data.csv') # Reading data in order to provide actual data as input
data.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,2015-07-11,2015-07-12,2015-07-13,2015-07-14,2015-07-15,2015-07-16,2015-07-17,2015-07-18,2015-07-19,2015-07-20,2015-07-21,2015-07-22,2015-07-23,2015-07-24,2015-07-25,2015-07-26,2015-07-27,2015-07-28,2015-07-29,2015-07-30,2015-07-31,2015-08-01,2015-08-02,2015-08-03,2015-08-04,2015-08-05,2015-08-06,2015-08-07,2015-08-08,...,2017-08-02,2017-08-03,2017-08-04,2017-08-05,2017-08-06,2017-08-07,2017-08-08,2017-08-09,2017-08-10,2017-08-11,2017-08-12,2017-08-13,2017-08-14,2017-08-15,2017-08-16,2017-08-17,2017-08-18,2017-08-19,2017-08-20,2017-08-21,2017-08-22,2017-08-23,2017-08-24,2017-08-25,2017-08-26,2017-08-27,2017-08-28,2017-08-29,2017-08-30,2017-08-31,2017-09-01,2017-09-02,2017-09-03,2017-09-04,2017-09-05,2017-09-06,2017-09-07,2017-09-08,2017-09-09,2017-09-10
0,5566_zh.wikipedia.org_all-access_spider,12.0,7.0,4.0,5.0,20.0,8.0,5.0,17.0,24.0,7.0,12.0,11.0,7.0,9.0,6.0,10.0,8.0,13.0,3.0,14.0,4.0,9.0,14.0,10.0,8.0,3.0,74.0,17.0,8.0,6.0,9.0,3.0,10.0,21.0,9.0,5.0,3.0,4.0,1.0,...,41.0,35.0,18.0,18.0,15.0,16.0,13.0,14.0,29.0,18.0,28.0,21.0,10.0,19.0,13.0,8.0,54.0,19.0,14.0,41.0,32.0,46.0,31.0,29.0,31.0,28.0,17.0,16.0,8.0,13.0,13.0,13.0,45.0,4.0,13.0,20.0,18.0,17.0,14.0,11.0
1,Intel_80386_zh.wikipedia.org_all-access_spider,7.0,5.0,9.0,3.0,5.0,1.0,6.0,6.0,3.0,3.0,0.0,1.0,2.0,3.0,6.0,7.0,11.0,8.0,7.0,8.0,9.0,11.0,11.0,11.0,2.0,2.0,7.0,9.0,7.0,6.0,1.0,1.0,8.0,3.0,5.0,6.0,7.0,3.0,2.0,...,13.0,8.0,16.0,15.0,7.0,14.0,7.0,13.0,7.0,6.0,6.0,3.0,15.0,15.0,15.0,8.0,18.0,15.0,10.0,10.0,4.0,27.0,21.0,16.0,5.0,11.0,13.0,8.0,2.0,10.0,7.0,7.0,10.0,9.0,2.0,6.0,7.0,9.0,6.0,5.0
2,Kara_zh.wikipedia.org_all-access_spider,56.0,25.0,7.0,50.0,90.0,30.0,19.0,16.0,10.0,12.0,13.0,19.0,10.0,10.0,22.0,55.0,14.0,6.0,22.0,26.0,16.0,23.0,24.0,29.0,52.0,10.0,4.0,8.0,13.0,16.0,13.0,10.0,12.0,11.0,14.0,6.0,6.0,6.0,3.0,...,20.0,13.0,12.0,44.0,18.0,7.0,5.0,16.0,7.0,22.0,8.0,18.0,13.0,17.0,17.0,10.0,13.0,13.0,22.0,14.0,15.0,34.0,16.0,16.0,19.0,16.0,23.0,19.0,8.0,11.0,13.0,13.0,11.0,9.0,9.0,14.0,19.0,36.0,10.0,8.0
3,MAMAMOO_zh.wikipedia.org_all-access_spider,8.0,7.0,17.0,11.0,8.0,1.0,1.0,10.0,6.0,8.0,2.0,2.0,3.0,8.0,5.0,4.0,4.0,11.0,6.0,6.0,10.0,10.0,8.0,5.0,7.0,2.0,3.0,6.0,8.0,13.0,5.0,5.0,25.0,2.0,10.0,3.0,2.0,6.0,4.0,...,25.0,6.0,7.0,6.0,8.0,10.0,7.0,5.0,8.0,10.0,9.0,12.0,6.0,11.0,10.0,10.0,14.0,21.0,27.0,20.0,23.0,38.0,29.0,10.0,13.0,19.0,13.0,24.0,10.0,12.0,9.0,19.0,17.0,19.0,8.0,4.0,10.0,10.0,10.0,11.0
4,Netflix_zh.wikipedia.org_all-access_spider,7.0,10.0,21.0,2.0,9.0,13.0,7.0,6.0,7.0,23.0,22.0,8.0,15.0,10.0,11.0,9.0,14.0,9.0,12.0,5.0,13.0,16.0,6.0,9.0,10.0,11.0,7.0,8.0,20.0,7.0,7.0,9.0,6.0,11.0,8.0,6.0,8.0,2.0,8.0,...,40.0,39.0,31.0,41.0,40.0,59.0,36.0,45.0,50.0,46.0,36.0,32.0,40.0,47.0,32.0,25.0,31.0,45.0,20.0,27.0,35.0,47.0,28.0,30.0,22.0,26.0,54.0,42.0,11.0,24.0,23.0,37.0,38.0,21.0,35.0,29.0,35.0,25.0,24.0,22.0


In [11]:
final_object.final_fun_2([(5,'2015-09-07',data.at[5,'2015-09-07']),
                         (265,'2016-01-09',data.at[265,'2016-01-09']), #Providing a list of 4 inputs
                         (1125,'2016-04-23',data.at[1125,'2016-04-23']),
                         (20,'2015-10-07',data.at[20,'2015-10-07'])])

+------------+------------+----------+-----------+--------+-------+----------------+
|   Client   |   Access   | Language | Predicted | Actual | SMAPE |   Time Taken   |
+------------+------------+----------+-----------+--------+-------+----------------+
|   spider   | all_access | Chinese  |     7     |   10   | 0.353 | 0:00:00.304356 |
| non-spider |  desktop   |  French  |    115    |  109   | 0.054 | 0:00:00.033399 |
| non-spider |   mobile   | Russian  |    308    |  227   | 0.303 | 0:00:00.031778 |
|   spider   | all_access | Chinese  |     3     |   4    | 0.286 | 0:00:00.034742 |
+------------+------------+----------+-----------+--------+-------+----------------+
